In [1]:
# sklearn imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# other imports
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
## Reading the Parquet file and converting it to a pandas dataframe
df = pq.read_table('CaoYouSample.parquet')
df = df.to_pandas()

In [9]:
## Data Cleaning steps before we jump into test - train split and transformations
df_new = df.copy()
rem_cols = ['CONM', 'TIC', 'CUSIP','FiscalYearEnd', 'FYEND_plus_3mos','LPERMNO','FYR','SIC','DATADATE']
df_new.drop(rem_cols, axis = 1, inplace = True)
df_new = df_new.drop_duplicates()
df_new = df_new.dropna()

In [17]:
df_new.columns

Index(['GVKEY', 'FYEAR', 'ACT', 'AP', 'AT', 'CEQ', 'CHE', 'DLC', 'DLTT',
       'INTAN', 'INVT', 'IVAO', 'LCT', 'LT', 'PPENT', 'RECT', 'TXP', 'COGS',
       'DP', 'DVC', 'NOPIO', 'SALE', 'TXT', 'XAD', 'XIDO', 'XINT', 'XRD',
       'XSGA', 'CSHO', 'SHRCD', 'EXCHCD', 'E', 'CFO', 'E_F1', 'SALE_D1',
       'COGS_D1', 'XSGA_D1', 'XAD_D1', 'XRD_D1', 'DP_D1', 'XINT_D1',
       'NOPIO_D1', 'TXT_D1', 'XIDO_D1', 'E_D1', 'DVC_D1', 'CHE_D1', 'INVT_D1',
       'RECT_D1', 'ACT_D1', 'PPENT_D1', 'IVAO_D1', 'INTAN_D1', 'AT_D1',
       'AP_D1', 'DLC_D1', 'TXP_D1', 'LCT_D1', 'DLTT_D1', 'LT_D1', 'CEQ_D1',
       'CFO_D1'],
      dtype='object')

In [10]:
### Writing a class for the eda of the data frame
class Panel_Data_Eda:
    
    ## Constructor of the class
    def __init__(self,df):
        self.df = df
    
    ## function to print the name of the data set
    def print_shape(self):
        print("The dimension of the data is: ", self.df.shape)
        
    
    ## Function to bucket all the columns of the dataset into their datatypes
    def col_classification(self):
        emp_dict = {}
        
        ## Initializing type dictionary
        for types in self.df.dtypes.unique():
            if types in emp_dict:
                continue
            else:
                emp_dict[types] = []
        
        # Appending column name to each data type 
        for col in self.df.columns:
            emp_dict[self.df[col].dtypes].append(col)
        
        return emp_dict
    
    ## Function to do test train split of panel data
    def test_train_splitter(self, cluster_col, time_col, window_width = 1):
        """
        -- The test train splitter for the 
        
        """
        gr1 = self.df.groupby([cluster_col], sort=False)
        df2= gr1.apply(lambda x: x.sort_values([time_col], ascending=False))
        df2 = df2.reset_index(drop=True)
        test = df2.groupby(cluster_col).head(window_width)
        jdf = pd.merge(df2, test, how='outer', indicator=True)
        train = jdf.loc[jdf['_merge'] == 'left_only']
        del train['_merge'], jdf
        return train,test
    
    ## Function to plot Univariate Time Series Data  
    def univariate_time_series_plot(self, name, xaxis, yaxis, cluster):

        """

        -- This function takes in the object and spits out Univariate plots for the specified sets of columns
        -- The parameter list is given below
            * self: The object
            * name: This variable contains the filteration criteria
            * xaxis: The vairable that will make the x axis of the univariate time series plot (mostly time)
            * yaxis: Variable that will make the y axis of the plot
            * cluster: This is the column which contains clusters

        """

        try:
            samp_df = self.df.loc[self.df[cluster] == name,:]
            if samp_df.shape[0] == 0:
                print("No rows left in the data frame after filteration. The function will return None")
                return None
            else:
                title_text = "Trend of " + yaxis + " for " + name
                fig = px.line(samp_df, x=xaxis, y=yaxis, title = title_text)
                return fig

        except ValueError:
            print("The value x/y column does not exist in the data frame")

        except NameError:
            print("The data frame that you are passing does not exist. Please review the input")

        # Except Clause 
        except KeyError:
            print("The column containing the filteration criteria does not exist in the data frame")
    
    
    ## Function to plot multiple trend lines (For various metrics) in a single plot   
    def multivariate_timeseries_plots(self, comp_name, col_lst, date_column, cluster):
        
        # Initializing the figure object
        fig = go.Figure()
        df = self.df.copy()
        samp_df = df.loc[df[cluster] == comp_name]
        
        ## iterating through all the items of the column list
        for item in col_lst:
            fig.add_trace(go.Scatter(
            x = samp_df[date_column],
            y = samp_df[item],
            mode = 'lines+markers',
            name = item
            ))
        
        # Adding the title
        fig.update_layout(title = ','.join(col_lst)+' for '+comp_name)
        
        # returning the figure
        return fig
    
    
    # print the type of column
    def column_type(self):
        dict_type = {}
        
        for col in self.df.columns:
            if self.df[col].dtypes not in dict_type.keys():
                dict_type[self.df[col].dtypes] = 1
            else:
                dict_type[self.df[col].dtypes] += 1
        
        col_class = self.col_classification()
        print('\n**********************************\n')
        print(col_class)
        print('\n**********************************\n')
        print(dict_type)

In [79]:
# Custom Transformer that extracts columns passed as argument to its constructor

class FeatureSelector(BaseEstimator, TransformerMixin):
    
    # Class constructor
    def __init__(self, feature_names):
        print("Feature Constructor was called")
        self.feature_names = feature_names
    
    # Return self, nothing else to do here
    def fit(self, X, y = None):
        print("Feature Selector fit method was called")
        return self
    
    # Method that describes what we need this transformer to do
    def transform(self, X, y = None):
        print("Feature selector transform method was called")
        return X[self.feature_names]


# Custom transformer for feature engineering
class NumericalTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        print("Numerical Transformer Constructor called")
        return None
    
    def fit(self, X, y = None):
        print("Numerical Transformer fit method was called")
        return self
    
    def transform(self, X, y= None):
        
        print("Numerical Transformer transform method was called")
        X_ = X.copy()
        for col in X_.columns:
            if col != 'CEQ':
                col_str = col + '_PER_EQ'
                # creating a new column
                X_.loc[:,col_str] = X_[col] / X_['CEQ']
                
                # dropping the redundant column
                X_.drop(col, axis = 1, inplace = True)
            
            else:
                continue
        
        X_ = X_.replace([np.inf, -np.inf], np.nan)
        X_ = X_.fillna(0)
        return X_.values

In [12]:
## Making the EDA object
obj2 = Panel_Data_Eda(df_new)

In [13]:
dtype_dict = obj2.col_classification()

In [34]:
X = [item for item in dtype_dict[list(dtype_dict.keys())[1]] if item != 'E_F1' and item != 'FYEAR']
y = [item for item in dtype_dict[list(dtype_dict.keys())[1]] if item == 'E_F1']

In [52]:
year = 2016

train_X = df_new.loc[(df_new['FYEAR'] >= (year-10)) & (df_new['FYEAR'] <= (year-1)), X]
train_y = df_new.loc[(df_new['FYEAR'] >= (year-10)) & (df_new['FYEAR'] <= (year-1)), y]

test_X = df_new.loc[df_new['FYEAR'] == year, X]
test_y = df_new.loc[df_new['FYEAR'] == year, y]

In [80]:
# Creating the transformation pipeline
# Numerical Features to pass down the numerical pipeline
numerical_features = X

# Defining the steps in the numerical pipeline
numerical_pipeline = Pipeline(steps = [
    ('num_selector', FeatureSelector(numerical_features)),
    ('num_transformer', NumericalTransformer())
])


# Preparing the data for the model
X_prepared = numerical_pipeline.fit_transform(train_X)

Feature Constructor was called
Numerical Transformer Constructor called
Feature Selector fit method was called
Feature selector transform method was called
Numerical Transformer fit method was called
Numerical Transformer transform method was called


In [ ]:
# Setting up the hyperparameter search space for the RF Model
parameters = {'max_features':['auto'],'max_depth':[20,25,30,35],'min_samples_leaf':[15,20,25,50]}
#parameters = {'max_features':['auto'],'max_depth':[20],'min_samples_leaf':[15]}

# Setting up the Random forest model
rf_mod = RandomForestRegressor(n_estimators=500,criterion='mse',oob_score=True,n_jobs=-1,random_state=10)

# Setting up the Grid Search object
grid_search = GridSearchCV(rf_mod, parameters, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Fitting the model
grid_search.fit(X_prepared, train_y)

In [82]:
test_X_prepared = numerical_pipeline.fit_transform(test_X)
pred = grid_search.predict(test_X_prepared)

Feature Selector fit method was called
Feature selector transform method was called
Numerical Transformer fit method was called
Numerical Transformer transform method was called


In [83]:
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, pred))}\n")

RMSE: 26.315216182167962

